# COVID-19 Dashboard

## A Global Pandemic (2019 - 2022)

In December 2019, the novel coronavirus SARS-CoV-2 was detected in Wuhan City, China (CDC, 2020), and its discovery altered the course of human history. The virus is very contagious and can cause a disease known as COVID-19 (coronavirus disease 2019). 

Although most people who have COVID-19 only experience mild symptoms, some people become extremely sick. Those who develop severe symptoms of the disease are often admitted to the hospital. 

Unfortunately, in some cases of severe infection, patients are unable to recover from the disease. In the United Kingdom alone, more than 200,000 deaths have been associated with COVID-19 (Public Health England, 2022). 

Due to the possibility of severe complications from contracting COVID-19, many governments around the world rushed to implement stay-at-home measures to protect their residents. While the effectiveness of these measures is not universally agreed upon, they did undoubtedly reduce the potential negative impact of the disease.

Thankfully, the world seems to be getting back to normal. Legal restrictions have been lifted in the United Kingdom and elsewhere, and society is quickly adjusting to the reality of post-pandemic life.

## Understanding the Statistics

Of course, there is incredible controversy over how the world responded to COVID-19. Reflection is an important process that can point out both the strengths and weaknesses of past actions.

Quantiative analysis is one of the most poignant ways to approach examining this unprecedented period in human history. Over the past few years, extensive data has been captured about the pandemic. Visualising that data paints a clearer picture of how the pandemic has developed over time.

## Graph One: New Cases vs New Tests

In the first graph pictured below, two statistics are compared: the number of new cases (by publish date) and the number of new tests (by publish date). Both of these statistics are updated on a daily basis.

Although new case numbers grew in the beginning of 2022, the graph shows that new case numbers have since returned to early-pandemic levels. The same trend applies to new test numbers. Where the two statistics differ is in the amount of zero-values. While daily test numbers are almost always above zero, the same is not the case with daily new case numbers. 

To refresh the data presented in the graph, click the "refresh" button underneath it. Changes will show when the graph is interacted with using the controls.

In [8]:
from IPython.display import clear_output 
from uk_covid19 import Cov19API 
import ipywidgets as wdg 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json 

# sets the visual output of the graph so that it is larger and easier to read
%matplotlib inline 
plt.rcParams['figure.dpi'] = 100

In [9]:
# This block reads the contents of JSON files and assigns them to variable names.

# returns the contents of a JSON file
def contentsOf(fileName: str):
    with open(fileName, "rt") as INFILE:
        return json.load(INFILE)

# returns only the 'data' portion of a JSON file
def rawData(fileName: str):
    data = contentsOf(fileName)
    return data['data']

dataset_one = rawData("newCasesAndNewTests.json")
dataset_two = rawData("ukCasesAndAdmissions.json")

In [10]:
# This block performs the data wrangling necessary to create the graph.

# reads the 'date' value from the data and converts it into optimal format
def parse_date(datestring: str):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# sorts through and reformats the data for use in the graph
def wrangle_data(dataset: str, setColumns: list):
    # sorts data by date
    dates = [dictionary['date'] for dictionary in dataset]
    dates.sort()
    
    # sets start and end dates for the graph
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

    # sets index to date range and creates dataframe
    index = pd.date_range(startdate, enddate, freq='D')
    df = pd.DataFrame(index = index, columns = setColumns)

    # populates columns in the dataset
    for entry in dataset:
        date = parse_date(entry['date'])
        for column in setColumns:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column]!= None else 0.0
                df.loc[date, column] = value 

    # fills in remaining holes
    df.fillna(0.0, inplace = True)

    return df 

newCasesAndNewTestsdf = wrangle_data(dataset_one, ['new_cases', 'new_tests'])
ukCasesAndAdmissionsdf = wrangle_data(dataset_two, ['cases', 'admissions'])

In [11]:
# This block creates the first graph and adds interactive controls.

def graph_one(gcols: str, gscale: str):
    if gscale == 'linear':
        logscale = False 
    else:
        logscale = True 

    ncols = len(gcols)

    if ncols > 0:
        newCasesAndNewTestsdf[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print('Click to select data for graph')
        print('CTRL-Click to select more than one category') 

# sets values of series (column) selection
series_one = wdg.SelectMultiple(
    options = ['new_cases', 'new_tests'],
    value = ['new_cases', 'new_tests'],
    rows = 2,
    description = 'Statistics:',
    disable = False
)

# sets values of scale selection
scale_one = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:', 
    disabled = False
) 

# defines appearance of the graph controls using series and select variables
controls_one = wdg.HBox([series_one, scale_one])

# creates UI and organizes it within a box
graph_one = wdg.interactive_output(graph_one, {'gcols': series_one, 'gscale': scale_one})

# outputs graph and interactive controls
display(controls_one, graph_one)

Output()

In [12]:
# This block creates a button that allows a user to refresh the data for graph one by polling the API. 

# accesses API to update the dataset
def access_api():
    # sets filter
    united_kingdom = [
        'areaType=overview'
    ]

    # sets structure
    new_cases_and_tests = {
        "date": "date",
        "new_cases": "newCasesByPublishDate",
        "new_tests": "newTestsByPublishDate"
    }

    # creates 'api' object
    api = Cov19API(filters=united_kingdom, structure=new_cases_and_tests)

    # polls the API for data
    data = api.get_json()

    return data['data']

# defines function of the button widget
def api_button_callback(button):
    apidata = access_api()

    global newCasesAndNewTestsdf

    # sets global dataframe variable to a wrangled version of the new dataset
    newCasesAndNewTestsdf = wrangle_data(apidata, ['new_cases', 'new_tests'])

    # sets visual appearance of the button
    apibutton.icon = "check"

apibutton = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Get new data from the API', 
    icon = 'check'
)

apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='info', description='Refresh Data', icon='check', style=ButtonStyle(), tooltip='Get new da…

## Graph Two: Total Cases vs Total Hospital Admissions

In the second graph, two additional statistics are compared: total cases (by specimen date) and total hospital admissions since the beginning of the pandemic. 

NOTE: Public Health England has changed the way it records total cases, so recent statistics may reflect a total of zero cumulative cases. 

Generally, both statistics follow a similar upward trend. Because both numbers will never decrease, it is more useful to compare them using a logarithmic scale. Doing so reveals that both case numbers and hospital admissions are growing at a relatively steady rate. Some could argue that this means COVID-19 is becoming more endemic, but it is likely too early to make such conclusions.

To refresh the data presented in the graph, click the "refresh" button underneath it. Changes will show when the graph is interacted with using the controls.

In [13]:
# This block creates the second graph and adds interactive controls.

def graph_two(gcols: str, gscale: str):
    if gscale == 'linear':
        logscale = False 
    else:
        logscale = True 

    ncols = len(gcols)

    if ncols > 0:
        ukCasesAndAdmissionsdf[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print('Click to select data for graph')
        print('CTRL-Click to select more than one category') 

# sets values of series (column) selection
series_two = wdg.SelectMultiple(
    options = ['cases', 'admissions'],
    value = ['cases', 'admissions'],
    rows = 2,
    description = 'Statistics:',
    disable = False
)

# sets values of scale selection
scale_two = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:', 
    disabled = False
) 

# defines appearance of the graph controls using series and select variables
controls_two = wdg.HBox([series_two, scale_two])

# creates UI and organizes it within a box
graph_two = wdg.interactive_output(graph_two, {'gcols': series_two, 'gscale': scale_two})

# outputs graph and interactive controls
display(controls_two, graph_two)

Output()

In [14]:
# This block creates a button that allows a user to refresh the data for graph two by polling the API. 

# accesses API to update the dataset
def access_api_two():
    # sets filter
    united_kingdom = [
        'areaType=overview'
    ]

    # sets structure
    cases_and_admissions = {
        "date": "date",
        "cases": "cumCasesBySpecimenDate",
        "admissions": "cumAdmissions"
    }

    # creates 'api' object
    api = Cov19API(filters=united_kingdom, structure=cases_and_admissions)

    # polls the API for data
    data = api.get_json()

    return data['data']

# defines function of the button widget
def api_button_callback_two(button):
    apidata = access_api_two()

    global ukCasesAndAdmissionsdf

    # sets global dataframe variable to a wrangled version of the new dataset
    ukCasesAndAdmissionsdf = wrangle_data(apidata, ['cases', 'admissions'])

    # sets visual appearance of the button
    apibutton_two.icon = "check"

apibutton_two = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Get new data from the API', 
    icon = 'check'
)

apibutton_two.on_click(api_button_callback_two)

display(apibutton_two)

Button(button_style='info', description='Refresh Data', icon='check', style=ButtonStyle(), tooltip='Get new da…

### References

CDC (2020) *COVID-19 and Your Health*. Available at: https://www.cdc.gov/coronavirus/2019-ncov/your-health/about-covid-19/basics-covid-19.html.

Public Health England (2022) *Coronavirus (COVID-19) in the UK*. Available at: https://coronavirus.data.gov.uk/ (Accessed: 1 December 2022).

#### Author and Copyright Notice

COVID-19 Dashboard (C) Matthew Streets, 2022 (hello@matthewstreets.com - web). All rights reserved.

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*